In [51]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import math
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestRegressor

In [52]:
#vérifie la version de panda
pd.__version__

'1.0.5'

In [53]:
# Definition des options et paramètres
pd.set_option("display.max_rows", None, "display.max_columns", None);
fileLocation = 'Dataset_3_sales.csv';
fileLocation2 = 'Dataset_3_sales_V2.csv';

In [54]:
# lecture du fichier csv et création d'un index
dfSales = pd.read_csv(fileLocation, sep="|",header=None);
dfSales.columns = dfSales.iloc[0, :].tolist();
dfSales = dfSales.drop(0, axis = 0).reset_index(drop = True);

#suppression des colonnes superflu
ID = dfSales.loc[:,'index'];
dfSales = dfSales.drop('index',axis = 1);
dfSales = dfSales.rename(columns={dfSales.columns[0]: "id"});
dfSales = dfSales.drop('id',axis =  1);
dfSales = dfSales.drop(dfSales.columns[[0,1,2,3,4,5,6,7]],axis =  1);

In [55]:
# Apercu de notre data set
dfSales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
1,DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
3,FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,NaN,Tier 3,Grocery Store,732.38
4,NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [56]:
# visualisation des données de la colonnes Item_Identifier
print(dfSales['Item_Identifier'].value_counts(dropna=False).sort_index())
print(dfSales['Item_Identifier'].value_counts(dropna=False).sum())

-400                    14
Allez au boulot ! :)    50
DRA12                    9
DRA24                   10
DRA59                   10
DRB01                    8
DRB13                    9
DRB24                    7
DRB25                   10
DRB48                    9
DRC01                    9
DRC12                    8
DRC13                    8
DRC24                    9
DRC25                   10
DRC27                   10
DRC36                   10
DRC49                    8
DRD01                    9
DRD12                    8
DRD13                    8
DRD15                   10
DRD24                   10
DRD25                   10
DRD27                    9
DRD37                    9
DRD49                    8
DRD60                    9
DRE01                    9
DRE03                    9
DRE12                    8
DRE13                   10
DRE15                   10
DRE25                    9
DRE27                    9
DRE37                    9
DRE48                    9
D

In [57]:
# Correction de la colonne Item_Identifier
# transformation des données sans rapport en 'XXXXXX'
dfSales['Item_Identifier'] = dfSales['Item_Identifier'].astype(str)
dfSales.loc[dfSales['Item_Identifier']=='-400','Item_Identifier']='XXXXX'
dfSales.loc[dfSales['Item_Identifier']=='Allez au boulot ! :)','Item_Identifier']='XXXXX'
dfSales.loc[dfSales['Item_Identifier']=='nan','Item_Identifier']='XXXXX'
dfSales['Item_Identifier'] = dfSales.loc[:,'Item_Identifier']

In [58]:
# vérification des modifications apporté
print(dfSales['Item_Identifier'].value_counts(dropna=False).sort_index())
print(dfSales['Item_Identifier'].value_counts(dropna=False).sum())

DRA12     9
DRA24    10
DRA59    10
DRB01     8
DRB13     9
DRB24     7
DRB25    10
DRB48     9
DRC01     9
DRC12     8
DRC13     8
DRC24     9
DRC25    10
DRC27    10
DRC36    10
DRC49     8
DRD01     9
DRD12     8
DRD13     8
DRD15    10
DRD24    10
DRD25    10
DRD27     9
DRD37     9
DRD49     8
DRD60     9
DRE01     9
DRE03     9
DRE12     8
DRE13    10
DRE15    10
DRE25     9
DRE27     9
DRE37     9
DRE48     9
DRE49     9
DRE60     9
DRF01    10
DRF03     9
DRF13     9
DRF15    10
DRF23     9
DRF25     9
DRF27     9
DRF36     9
DRF37     9
DRF48     9
DRF49     9
DRF51     8
DRF60     8
DRG01     9
DRG03     9
DRG11     9
DRG13     8
DRG15     8
DRG23     9
DRG25     8
DRG27     9
DRG36     8
DRG37    10
DRG39     8
DRG48    10
DRG49    10
DRG51     9
DRH01     9
DRH03    10
DRH11    10
DRH13     9
DRH15     9
DRH23    10
DRH25     9
DRH36     9
DRH37    10
DRH39    10
DRH49    10
DRH51     9
DRH59     9
DRI01     8
DRI03    10
DRI11    10
DRI13     8
DRI23     8
DRI25     9
DRI3

In [59]:
# visualisation des données de la colonnes Item_Weight
print(dfSales['Item_Weight'].value_counts(dropna=False).sort_index());
print(dfSales['Item_Weight'].value_counts(dropna=False).sum());

10.0             74
10.1             75
10.195           92
10.3             73
10.395           44
10.5            123
10.6             29
10.65            22
10.695           54
10.8             68
10.85            15
10.895           62
11.0             52
11.1             67
11.15            51
11.3             45
11.35            44
11.395           23
11.5             82
11.6             68
11.65            68
11.8            113
11.85            22
12.0             31
12.1             81
12.15           127
12.3             77
12.35            74
12.5             92
12.6             96
12.65            61
12.8             55
12.85            84
13.0             66
13.1             55
13.15            69
13.3             38
13.35            76
13.5             75
13.6             37
13.65           115
13.8             54
13.85            46
14.0             92
14.1             36
14.15            75
14.3             54
14.35            23
14.5             69
14.6             39


In [60]:
# Correction de la colonne Item_Identifier
#transformation des données non numériques en 0
dfSales['Item_Weight'] = dfSales['Item_Weight'].astype(str)
dfSales.loc[dfSales['Item_Weight']=='ù*ùfsfsf///','Item_Weight']=0.0
dfSales.loc[dfSales['Item_Weight']=='nan','Item_Weight']=0.0

In [61]:
# vérification des modifications apporté
dfSales['Item_Weight'] = dfSales['Item_Weight'].astype(float)
print(dfSales['Item_Weight'].value_counts(dropna=False).sort_index());
print(dfSales['Item_Weight'].value_counts(dropna=False).sum());

0.000     2449
4.555        7
4.590        7
4.610        8
4.615        6
4.635        7
4.785        8
4.805        7
4.880        8
4.905        7
4.920        7
5.000        8
5.030        8
5.035        8
5.095        8
5.110        8
5.150        8
5.155        7
5.175       24
5.190       14
5.210        7
5.260       16
5.305        8
5.320        7
5.325        8
5.340        8
5.365        8
5.400        8
5.405        7
5.425        8
5.440       15
5.460       16
5.465        7
5.480        8
5.485        8
5.500        8
5.510        7
5.590        8
5.615        8
5.630        8
5.635        8
5.655       16
5.675        7
5.695       16
5.730       16
5.735        8
5.750        7
5.765       14
5.780       23
5.785       23
5.800        7
5.820       15
5.825        8
5.845        8
5.860        7
5.880       28
5.885        7
5.905        7
5.920        8
5.925        7
5.940       15
5.945        8
5.980       16
5.985       14
6.030        8
6.035       16
6.055     

In [62]:
# vérification des modifications apporté
dfSales['Item_Weight'].value_counts()

0.000     2449
17.600     135
12.150     127
10.500     123
13.650     115
11.800     113
15.100     111
19.350     109
15.700     102
15.600     100
16.700     100
17.750      99
9.800       99
20.250      97
12.600      96
16.000      95
15.850      93
9.300       93
17.700      93
18.850      92
14.000      92
12.500      92
10.195      92
20.700      92
15.000      91
19.700      91
17.850      89
18.250      89
16.750      89
9.600       85
12.850      84
11.500      82
9.500       82
9.195       82
20.200      82
12.100      81
19.200      79
15.200      78
12.300      77
15.500      77
19.600      76
13.350      76
17.250      76
16.200      75
20.350      75
10.100      75
14.150      75
13.500      75
16.500      74
10.000      74
12.350      74
15.350      74
9.000       74
10.300      73
20.500      72
19.000      71
16.100      70
14.500      69
8.895       69
13.150      69
16.350      69
10.800      68
18.350      68
20.750      68
11.650      68
11.600      68
18.600    

In [63]:
# visualisation des données de la colonnes Item_Fat_Content
print(dfSales['Item_Fat_Content'].value_counts(dropna=False));
print(dfSales['Item_Fat_Content'].value_counts(dropna=False).sum())

Low Fat    8443
Regular    4803
LF          521
reg         195
low fat     177
NaN          50
None         15
Name: Item_Fat_Content, dtype: int64
14204


In [64]:
# Correction de la colonne Item_Fat_Content
#remplacement des valeurs 'low fat' et 'LF' en 'Low Fat' afin de réduire le nombre de catégorie
dfSales['Item_Fat_Content'] = dfSales['Item_Fat_Content'].astype(str)
dfSales['Item_Fat_Content'] = dfSales['Item_Fat_Content'].replace(['low fat', 'LF'], 'Low Fat')
#remplacement des valeurs 'reg' en 'Regular'
dfSales['Item_Fat_Content'] = dfSales['Item_Fat_Content'].replace(['reg'], 'Regular')
#remplacement des valeurs 'nan' en 'None'
dfSales.loc[dfSales['Item_Fat_Content']=='nan','Item_Fat_Content']='None'

In [65]:
# vérification des modifications apporté
print(dfSales['Item_Fat_Content'].value_counts(dropna=False));
print(dfSales['Item_Fat_Content'].value_counts(dropna=False).sum())

Low Fat    9141
Regular    4998
None         65
Name: Item_Fat_Content, dtype: int64
14204


In [66]:
# visualisation des données de la colonne Item_Visibility
print(dfSales['Item_Visibility'].value_counts(sort=True));

0.0                      879
0.076841095                3
0.077168705                3
0.076855628                3
0.076975118                3
0.076483451                3
0.07679167099999999        3
0.077290355                3
0.077011493                3
0.092933158                2
0.135620265                2
0.079739853                2
0.046595951                2
0.037574137                2
0.071948252                2
0.279650277                2
0.087272074                2
0.026835691000000002       2
0.05813220700000001        2
0.13689554                 2
0.041291168999999996       2
0.056191301                2
0.037793818                2
0.157149885                2
0.072139167                2
0.08486204                 2
0.026882496000000002       2
0.041355366                2
0.103422709                2
0.071992202                2
0.09725893699999999        2
0.041009558                2
0.106968096                2
0.026840766000000002       2
0.058778429000

In [67]:
# Transformation des données de la colonne en float
dfSales['Item_Visibility'] = dfSales['Item_Visibility'].astype(float)

In [68]:
# vérification des modifications apporté
print(dfSales['Item_Visibility'].value_counts(sort=True));

0.000000    879
0.076841      3
0.077011      3
0.076792      3
0.076856      3
0.076975      3
0.077290      3
0.077169      3
0.076483      3
0.159721      2
0.085204      2
0.058827      2
0.112682      2
0.093531      2
0.046729      2
0.086765      2
0.121227      2
0.121521      2
0.127416      2
0.159690      2
0.120663      2
0.187841      2
0.079420      2
0.037794      2
0.103442      2
0.058778      2
0.034875      2
0.074779      2
0.080250      2
0.060091      2
0.059557      2
0.058461      2
0.134564      2
0.122016      2
0.080640      2
0.157719      2
0.037657      2
0.127930      2
0.058084      2
0.072445      2
0.044945      2
0.279650      2
0.045237      2
0.080456      2
0.072707      2
0.071948      2
0.107110      2
0.060185      2
0.056306      2
0.037574      2
0.080278      2
0.079740      2
0.103603      2
0.072238      2
0.103423      2
0.073859      2
0.156072      2
0.040946      2
0.024795      2
0.072317      2
0.097457      2
0.041525      2
0.024656

In [69]:
# visualisation des données de la colonne Item_Type
print(dfSales['Item_Type'].value_counts());
print(dfSales['Item_Type'].value_counts().sum());

Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64
14204


In [70]:
# visualisation et transformation des données de la colonne Item_MRP en float
dfSales['Item_MRP'] = dfSales['Item_MRP'].astype(float)
print(dfSales['Item_MRP'].value_counts())
print(dfSales['Item_MRP'].value_counts().sum())

172.0422    8
127.1020    7
179.7660    7
177.9370    7
177.0370    7
196.5768    7
110.1544    7
182.1608    7
123.1046    7
178.2370    7
166.0526    7
178.4370    7
175.0370    7
113.2834    7
188.1872    7
113.1544    7
88.9856     6
102.7332    6
160.9578    6
196.5084    6
152.8366    6
37.8506     6
170.5422    6
181.7634    6
143.2154    6
160.6578    6
142.0154    6
125.2678    6
50.3692     6
45.9060     6
115.1834    6
177.6370    6
102.8990    6
189.0530    6
141.2154    6
154.4630    6
97.0094     6
121.3098    6
140.6154    6
194.7110    6
101.7990    6
195.3478    6
165.9526    6
127.9678    6
62.3510     6
100.1016    6
156.5288    6
55.6614     6
109.5228    6
119.8124    6
37.9532     6
120.8756    6
107.4622    6
189.6214    6
150.7366    6
108.6912    6
198.2110    6
57.0614     6
59.2562     6
94.4094     6
139.9838    6
192.2478    6
144.6102    6
138.6838    5
162.5526    5
125.5046    5
239.4538    5
119.1440    5
129.2626    5
89.8856     5
88.5198     5
94.712

In [71]:
# visualisation des données de la colonne Outlet_Identifier
print(dfSales['Outlet_Identifier'].value_counts())
print(dfSales['Outlet_Identifier'].value_counts().sum())

OUT027    1559
OUT013    1553
OUT049    1550
OUT035    1550
OUT046    1550
OUT045    1548
OUT018    1546
OUT017    1543
OUT010     925
OUT019     880
Name: Outlet_Identifier, dtype: int64
14204


In [72]:
# visualisation et transformation des données de la colonne Outlet_Establishment_Year en int
dfSales['Outlet_Establishment_Year'] = dfSales['Outlet_Establishment_Year'].astype(int)
print(dfSales['Outlet_Establishment_Year'].value_counts())
print(dfSales['Outlet_Establishment_Year'].value_counts().sum())

1985    2439
1987    1553
1999    1550
1997    1550
2004    1550
2002    1548
2009    1546
2007    1543
1998     925
Name: Outlet_Establishment_Year, dtype: int64
14204


In [73]:
# visualisation et transformation des données de la colonne Outlet_Size en str
dfSales['Outlet_Size'] = dfSales['Outlet_Size'].astype(str)
print(dfSales['Outlet_Size'].value_counts(dropna=False))
print(dfSales['Outlet_Size'].value_counts(dropna=False).sum())

Medium    4655
nan       4016
Small     3980
High      1553
Name: Outlet_Size, dtype: int64
14204


In [74]:
# transformation des valeurs 'nan' en 'None'
dfSales.loc[dfSales['Outlet_Size']=='nan','Outlet_Size']='None'

In [75]:
# vérification des modifications apporté
print(dfSales['Outlet_Size'].value_counts())
print(dfSales['Outlet_Size'].value_counts().sum())

Medium    4655
None      4016
Small     3980
High      1553
Name: Outlet_Size, dtype: int64
14204


In [76]:
# visualisation et transformation des données de la colonne Outlet_Location_Type en str
dfSales['Outlet_Location_Type'] = dfSales['Outlet_Location_Type'].astype(str)
print(dfSales['Outlet_Location_Type'].value_counts())
print(dfSales['Outlet_Location_Type'].value_counts().sum())

Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64
14204


In [77]:
# visualisation et transformation des données de la colonne Outlet_Type en str
dfSales['Outlet_Type'] = dfSales['Outlet_Type'].astype(str)
print(dfSales['Outlet_Type'].value_counts())
print(dfSales['Outlet_Type'].value_counts().sum())

Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64
14204


In [78]:
# visualisation et transformation des données de la colonne Item_Outlet_Sales en str
dfSales['Item_Outlet_Sales'] = dfSales['Item_Outlet_Sales'].astype(str)
print(dfSales['Item_Outlet_Sales'].value_counts(dropna=False).sort_index())
print(dfSales['Item_Outlet_Sales'].value_counts(dropna=False).sum())

1000.6974                1
1001.3632                5
1004.0264                4
1006.6896               10
1007.3554                1
10072.8882               1
101.2016                 4
101.8674                 2
1010.6844                2
1011.3502                1
1012.0160000000001       4
1012.6818                1
1014.6792                2
1015.345                 1
1017.3424                4
1018.0082                3
1018.674                 8
102.5332                 3
1020.6714                1
1021.3372                1
1022.6688                7
1023.3346                1
10236.675                1
1024.6662                4
1025.3319999999999       4
1025.9978                1
10256.649                1
1029.9926                2
103.199                  3
1030.6584                5
10306.584                1
1031.99                  4
1032.6558                1
1033.3216                1
1034.6532                6
1035.319                 1
1035.9848                2
1

In [79]:
# transformation des valeurs 'nan' en valeurs '0'
dfSales.loc[dfSales['Item_Outlet_Sales']=='nan','Item_Outlet_Sales']='0'

In [80]:
#retransformation des valeurs de la colonne en float et vérification des modifications
dfSales['Item_Outlet_Sales'] = dfSales['Item_Outlet_Sales'].astype(float)
print(dfSales['Item_Outlet_Sales'].value_counts(dropna=False).sort_index())
print(dfSales['Item_Outlet_Sales'].value_counts(dropna=False).sum())

0.0000        5681
33.2900          2
33.9558          1
34.6216          1
35.2874          1
36.6190          2
37.2848          1
37.9506          5
38.6164          2
39.9480          2
40.6138          2
41.2796          3
41.9454          3
42.6112          2
44.6086          2
45.2744          1
45.9402          2
47.2718          3
47.9376          2
48.6034          2
49.2692          2
49.9350          1
50.6008          3
51.9324          1
53.2640          1
53.9298          1
55.2614          1
56.5930          1
57.2588          1
58.5904          1
59.2562          1
61.2536          2
61.9194          1
63.2510          1
65.2484          1
67.9116          3
69.2432          1
70.5748          1
71.9064          2
73.2380          3
74.5696          1
75.2354          1
75.9012          4
77.2328          1
78.5644          2
79.2302          1
79.8960          3
80.5618          1
81.2276          1
82.5592          1
83.2250          1
83.8908          3
85.8882     

In [81]:
#suppression des lignes contenant des 'None' celles ci ne permettant pas d'apporter d'information sur le produit
indexname=dfSales[dfSales['Item_Fat_Content'] == 'None' ].index
dfSales.drop(indexname , inplace = True)

In [82]:
#vérification des modifications apportés
print(dfSales['Item_Fat_Content'].value_counts(dropna=False));
print(dfSales['Item_Fat_Content'].value_counts(dropna=False).sum())

Low Fat    9141
Regular    4998
Name: Item_Fat_Content, dtype: int64
14139


In [83]:
#suppression des lignes contenant la valeur 'XXXXX' celles ci ne permettant pas d'identifier le produit
indexname2=dfSales[dfSales['Item_Identifier'] == 'XXXXX' ].index
dfSales.drop(indexname2 , inplace = True)

In [84]:
#vérification des modifications apportés
print(dfSales['Item_Identifier'].value_counts(dropna=False));
print(dfSales['Item_Identifier'].value_counts(dropna=False).sum())

FDZ48    10
FDB17    10
FDQ28    10
FDK15    10
FDL58    10
FDP11    10
NCC06    10
FDL15    10
FDS08    10
NCJ17    10
FDM39    10
FDB41    10
NCN41    10
FDD16    10
FDS32    10
DRC36    10
FDU28    10
FDF56    10
FDJ41    10
NCJ06    10
NCF54    10
FDC53    10
FDR07    10
NCR30    10
FDX40    10
FDM60    10
FDP23    10
FDW39    10
NCY29    10
NCQ30    10
FDX03    10
DRD25    10
FDX08    10
FDT13    10
DRA24    10
FDZ26    10
FDJ02    10
FDU57    10
FDP38    10
FDS13    10
FDA50    10
NCZ54    10
FDW24    10
FDS47    10
FDM56    10
NCZ30    10
DRZ24    10
FDT39    10
FDH56    10
FDO31    10
NCE07    10
FDV01    10
NCE43    10
NCU18    10
NCG43    10
FDF52    10
FDO56    10
FDA49    10
FDZ28    10
FDJ57    10
FDE02    10
FDC21    10
NCP41    10
FDY21    10
FDT34    10
NCV06    10
FDU48    10
FDD05    10
FDY58    10
FDA07    10
FDK52    10
FDG10    10
FDI20    10
FDN38    10
FDO27    10
FDJ53    10
FDG09    10
FDP59    10
FDA52    10
FDD53    10
NCD43    10
NCM43    10
FDD38    10
NCO5

In [85]:
#suppression des lignes contenant des 0 celles ci ne permettant pas d'apporter d'information sur le produit
indexname3=dfSales[dfSales['Item_Weight'] == 0 ].index
dfSales.drop(indexname3 , inplace = True)
print(dfSales['Item_Weight'].value_counts(dropna=False));
print(dfSales['Item_Weight'].value_counts(dropna=False).sum())

17.600    133
12.150    126
10.500    123
13.650    115
11.800    112
15.100    110
19.350    109
16.700    100
15.700    100
15.600     99
17.750     99
9.800      98
20.250     97
12.600     96
16.000     93
9.300      93
12.500     92
10.195     92
18.850     91
17.700     91
20.700     91
15.000     90
19.700     90
15.850     90
14.000     90
17.850     88
18.250     88
16.750     87
9.600      85
12.850     83
11.500     82
20.200     82
9.500      81
9.195      80
12.100     80
19.200     79
15.200     78
15.500     77
13.350     76
12.300     76
16.200     75
13.500     75
20.350     75
19.600     75
17.250     75
14.150     74
10.100     74
12.350     74
10.300     73
10.000     72
20.500     72
15.350     72
9.000      72
16.500     72
16.100     70
11.600     68
8.895      68
14.500     68
19.000     68
11.650     68
16.350     68
13.150     68
18.350     67
10.800     67
20.750     66
18.700     66
9.395      66
18.200     66
18.600     66
11.100     66
19.850     65
18.000

In [86]:
#suppression des lignes contenant des 0 celles ci ne permettant pas d'apporter d'information sur le produit
indexname4=dfSales[dfSales['Item_Outlet_Sales'] ==0].index
dfSales.drop(indexname4 , inplace = True)
print(dfSales['Item_Outlet_Sales'].value_counts(dropna=False));
print(dfSales['Item_Outlet_Sales'].value_counts(dropna=False).sum())

958.7520      16
1342.2528     15
1230.3984     14
703.0848      14
1278.3360     13
1845.5976     13
1416.8224     13
539.2980      12
575.2512      11
852.2240      11
639.1680      11
1438.1280     11
1406.1696     11
1571.2880     10
1211.7560     10
2277.0360     10
527.3136      10
2759.0752     10
1701.7848     10
1537.9980     10
679.1160      10
1821.6288      9
1225.0720      9
1518.0240      9
1713.7692      9
2684.5056      9
2013.3792      9
1869.5664      9
2348.9424      9
2696.4900      9
1470.0864      9
2117.2440      9
1304.9680      8
1757.7120      8
1018.6740      8
1794.3310      8
2812.3392      8
2376.9060      8
605.8780       8
2693.8268      8
2109.2544      8
778.9860       8
452.7440       8
1853.5872      8
1090.5804      8
2816.3340      8
1258.3620      8
633.8416       8
2059.9852      8
759.0120       8
3020.0688      8
1151.1682      8
671.1264       8
1593.9252      8
2157.1920      8
2956.1520      8
2302.3364      8
1693.7952      8
1118.5440     

In [87]:
#modification des lignes contenant des 0 par la moyenne
dfSales.loc[dfSales['Item_Visibility']==0,'Item_Visibility']=dfSales.Item_Visibility.mean()
print(dfSales['Item_Visibility'].value_counts(dropna=False));
print(dfSales['Item_Visibility'].value_counts(dropna=False).sum())

0.063968    433
0.076975      3
0.037794      2
0.086765      2
0.037640      2
0.076856      2
0.162462      2
0.041178      2
0.058084      2
0.103442      2
0.121521      2
0.041113      2
0.044024      2
0.034868      2
0.071833      2
0.026818      2
0.080640      2
0.085622      2
0.159721      2
0.041291      2
0.058121      2
0.112399      2
0.026950      2
0.077290      2
0.112682      2
0.097611      2
0.103423      2
0.040912      2
0.052150      2
0.059836      2
0.128296      2
0.081097      2
0.135944      2
0.072253      2
0.107057      2
0.107224      2
0.079806      2
0.135708      2
0.024756      2
0.144920      2
0.061425      2
0.128012      2
0.074931      2
0.056191      2
0.014099      2
0.071958      2
0.074613      2
0.061271      2
0.076792      2
0.112859      2
0.134976      2
0.041283      2
0.058919      2
0.121250      2
0.074627      2
0.041355      2
0.112203      2
0.157076      2
0.037734      2
0.097457      2
0.061164      2
0.085275      2
0.160624

In [88]:
#sauvegarde des données précédentes dans un nouveau fichier csv
dfSales.to_csv('Dataset_3_sales_V2.csv',sep=";",header=True, index=False)

In [89]:
#création d'un nouveau dataframe
dfSalesv2 = pd.read_csv(fileLocation2, sep=";");

In [90]:
#créations de colonnes permettant de trier les valeurs des colonnes étant en str avec des identifiants
dfSalesv2.Item_Identifier=pd.Categorical(dfSalesv2.Item_Identifier)
dfSalesv2['ID_Objet'] = dfSalesv2.Item_Identifier.cat.codes + 1
#dfSalesv2['Item_Identifier']=dfSalesv2['ID_Objet']
#
dfSalesv2.Item_Fat_Content=pd.Categorical(dfSalesv2.Item_Fat_Content)
dfSalesv2['ID_Fat_Content'] = dfSalesv2.Item_Fat_Content.cat.codes + 1
#dfSalesv2['Item_Fat_Content']=dfSalesv2['ID_Fat_Content']
#
dfSalesv2.Item_Type=pd.Categorical(dfSalesv2.Item_Type)
dfSalesv2['ID_Item_Type'] = dfSalesv2.Item_Type.cat.codes + 1
#dfSalesv2['Item_Type']=dfSalesv2['ID_Item_Type']
#
dfSalesv2.Outlet_Identifier=pd.Categorical(dfSalesv2.Outlet_Identifier)
dfSalesv2['ID_Outlet_Identifier'] = dfSalesv2.Outlet_Identifier.cat.codes + 1
#dfSalesv2['Outlet_Identifier']=dfSalesv2['ID_Outlet_Identifier']
#
dfSalesv2.Outlet_Size=pd.Categorical(dfSalesv2.Outlet_Size)
dfSalesv2['ID_Outlet_Size'] = dfSalesv2.Outlet_Size.cat.codes + 1
#dfSalesv2['Outlet_Size']=dfSalesv2['ID_Outlet_Size']
#
dfSalesv2.Outlet_Location_Type=pd.Categorical(dfSalesv2.Outlet_Location_Type)
dfSalesv2['ID_Outlet_Location_Type'] = dfSalesv2.Outlet_Location_Type.cat.codes + 1
#dfSalesv2['Outlet_Location_Type']=dfSalesv2['ID_Outlet_Location_Type']
#
dfSalesv2.Outlet_Type=pd.Categorical(dfSalesv2.Outlet_Type)
dfSalesv2['ID_Outlet_Type'] = dfSalesv2.Outlet_Type.cat.codes + 1
#dfSalesv2['Outlet_Type']=dfSalesv2['ID_Outlet_Type']
#
dfSalesv2.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,ID_Objet,ID_Fat_Content,ID_Item_Type,ID_Outlet_Identifier,ID_Outlet_Size,ID_Outlet_Location_Type,ID_Outlet_Type
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,157,1,5,8,2,1,2
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,9,2,15,4,2,3,3
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,661,1,11,8,2,1,2
3,FDX07,19.200,Regular,0.063968,Fruits and Vegetables,182.0950,OUT010,1998,None,Tier 3,Grocery Store,732.3800,1118,2,7,1,3,3,1
4,NCD19,8.930,Low Fat,0.063968,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,1294,1,10,2,1,3,2
5,FDP36,10.395,Regular,0.063968,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,756,2,1,4,2,3,3
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,694,2,14,2,1,3,2
7,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,None,Tier 2,Supermarket Type1,1076.5986,440,2,6,6,3,2,2
8,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,None,Tier 2,Supermarket Type1,4710.5350,987,2,6,3,3,2,2
9,FDY07,11.800,Low Fat,0.063968,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266,1168,1,7,8,2,1,2


In [91]:
#restructuration du dataframe
dfSalesv2 = dfSalesv2[['Item_Identifier','ID_Objet',
'Item_Weight',
'Item_Fat_Content','ID_Fat_Content',
'Item_Visibility',
'Item_Type','ID_Item_Type',
'Item_MRP',
'Outlet_Identifier','ID_Outlet_Identifier',
'Outlet_Establishment_Year',
'Outlet_Size','ID_Outlet_Size',
'Outlet_Location_Type','ID_Outlet_Location_Type',
'Outlet_Type','ID_Outlet_Type',
'Item_Outlet_Sales']];

In [92]:
#visualisation du dataframe
dfSalesv2.head(10)

,Item_Identifier,ID_Objet,Item_Weight,Item_Fat_Content,ID_Fat_Content,Item_Visibility,Item_Type,ID_Item_Type,Item_MRP,Outlet_Identifier,ID_Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,ID_Outlet_Size,Outlet_Location_Type,ID_Outlet_Location_Type,Outlet_Type,ID_Outlet_Type,Item_Outlet_Sales
0,FDA15,157,9.300,Low Fat,1,0.016047,Dairy,5,249.8092,OUT049,8,1999,Medium,2,Tier 1,1,Supermarket Type1,2,3735.1380
1,DRC01,9,5.920,Regular,2,0.019278,Soft Drinks,15,48.2692,OUT018,4,2009,Medium,2,Tier 3,3,Supermarket Type2,3,443.4228
2,FDN15,661,17.500,Low Fat,1,0.016760,Meat,11,141.6180,OUT049,8,1999,Medium,2,Tier 1,1,Supermarket Type1,2,2097.2700
3,FDX07,1118,19.200,Regular,2,0.063968,Fruits and Vegetables,7,182.0950,OUT010,1,1998,None,3,Tier 3,3,Grocery Store,1,732.3800
4,NCD19,1294,8.930,Low Fat,1,0.063968,Household,10,53.8614,OUT013,2,1987,High,1,Tier 3,3,Supermarket Type1,2,994.7052
5,FDP36,756,10.395,Regular,2,0.063968,Baking Goods,1,51.4008,OUT018,4,2009,Medium,2,Tier 3,3,Supermarket Type2,3,556.6088
6,FDO10,694,13.650,Regular,2,0.012741,Snack Foods,14,57.6588,OUT013,2,1987,High,1,Tier 3,3,Supermarket Type1,2,343.5528
7,FDH17,440,16.200,Regular,2,0.016687,Frozen Foods,6,96.9726,OUT045,6,2002,None,3,Tier 2,2,Supermarket Type1,2,1076.5986
8,FDU28,987,19.200,Regular,2,0.094450,Frozen Foods,6,187.8214,OUT017,3,2007,None,3,Tier 2,2,Supermarket Type1,2,4710.5350
9,FDY07,1168,11.800,Low Fat,1,0.063968,Fruits and Vegetables,7,45.5402,OUT049,8,1999,Medium,2,Tier 1,1,Supermarket Type1,2,1516.0266


In [93]:
#suppression des colonnes en str celles ci étant remplacé par des ID
dfSalesv2 = dfSalesv2[['ID_Objet',
'Item_Weight',
'ID_Fat_Content',
'Item_Visibility',
'ID_Item_Type',
'Item_MRP',
'ID_Outlet_Identifier',
'Outlet_Establishment_Year',
'ID_Outlet_Size',
'ID_Outlet_Location_Type',
'ID_Outlet_Type',
'Item_Outlet_Sales']];

In [94]:
#création d'un dataframe final
dfSalesv2.to_csv('Dataset_3_sales_V3b.csv',sep=";",header=True, index=False)


In [95]:
#analyse du dataframe par le profileReport de panda
from pandas_profiling import ProfileReport
ProfileReport(dfSalesv2)

ModuleNotFoundError: No module named 'pandas_profiling'

In [96]:
#selection des colonnes qui serviront à la prédiction
X = dfSalesv2[['Item_Weight','ID_Fat_Content','Item_Visibility','ID_Item_Type','Item_MRP','ID_Outlet_Size','ID_Outlet_Location_Type','ID_Outlet_Type','Outlet_Establishment_Year',]]
y = dfSalesv2['Item_Outlet_Sales']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [98]:
#tentative de prédiction en régression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import GradientBoostingRegressor

randomforest=GradientBoostingRegressor(n_estimators=200) 
y=dfSalesv2['Item_Outlet_Sales']
x=dfSalesv2.loc[:,dfSalesv2.columns !='Item_Outlet_Sales']
#print(x)
randomforest.fit(x,y)
score=cross_val_score(randomforest,x,y,cv=15)
moyenne=score.mean()
y_predit=randomforest.predict(x) 
print(moyenne)

0.5056590630971741
